In [ ]:
from imports import *

def update_key_value(a, b):
    '''updating key & values (values will be appened for existing key)'''
    c = {}
    tls = lambda ls : ls if type(ls) is list else [ls]
    for k, v in a.items():
        c[k] = tls(a[k]) + tls(b[k]) if k in list(b) else tls(a[k])
    else: return {**b, **c}


class WhBase:
    
    pane_search = "/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]"
    body_chat_search_btn = "//div[@id='main']//div[@class='_3ndVb']"
    body_chat_name = "//div[@id='main']//div[@class='_21nHd']"
    body_text_area = "//footer[@class='_3E8Fg']//p"
    body_msg_nav_arrow = '/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[1]/span/div'
    init_info = "//div[@data-testid='intro-text'][contains(.,'Use WhatsApp on up to 4 linked devices and 1 phone at the same time.')]"
    pane_search = "//div[@id='side']//div[@data-testid='chat-list-search']"
    pane_search_back ="//div[@id='side']//span[@data-testid='search']"
    pane_search_cancel = "//div[@id='side']//span[contains(@data-testid,'x-alt')]"
    pane_search_filter = "//div[@id='side']//button[@aria-label='Unread chats filter']"
    active_chat_title = "//div[@id='main']//span[@data-testid='conversation-info-header-chat-title']"
    active_chat_search = "//div[@id='main']//span[@data-testid='search-alt']"

    
    def __init__(self):
        chrome_options = Options()
        chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        self.wait = WebDriverWait(self.driver, 5)
        self.ac = ActionChains(self.driver)
        self.read_store = defaultdict()
        self.store_msg = pd.DataFrame([])
        print(self.driver.title)
    
    @property
    def get_driver(self):
        return self.driver
    
    @property
    def reload_get(self):
        wait = WebDriverWait(self.driver, 60)
        self.driver.refresh()
        search_bar_xpt = "/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]"
        element = wait.until(EC.presence_of_element_located((By.XPATH, search_bar_xpt)))
        print('reload success, whatsapp ready')
        return self.driver
    
    def xpelem(self, xpt):
        try: return self.wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
        except: return None
    
    def xptext(self, xpt):
        try: return self.driver.find_element(By.XPATH, xpt).text
        except: return None
        
    def xpget(self, xpt):
        try: return self.driver.find_element(By.XPATH, xpt)
        except: return None
    
    def click(self, xpt):
        xp = self.xpelem(xpt) if type(xpt) is str else xpt
        if xp is None: 
            print("xpath not present")
            return None
        try:
            xp.click()
            return 1
        except: return 0
    
    def move_click(self, xpt):
        try: 
            self.ac.move_to_element(self.xpget(xpt)).click(xpt).perform()
            return 1
        except: return 0
    
    def clickable(self, xpt):
        try: return wait.until(EC.element_to_be_clickable((By.XPATH, xpt)))
        except: return None
        
    def xpds(self, xpt):
        x = self.xpelem(xpt)
        if x is None:
            print('xpath not found')
        else:
            print('xpath present')
            if self.clickable(xpt) is not None: 
                print('clickable')
            else: 
                print('not clickable')
            print('text: ', self.xptext(xpt))
        
    def hover_click(self, xp1, xp2):
        try: 
            self.ac.move_to_element(self.xpelem(xp1)).click(self.xpelem(xp2)).perform()
            return 1
        except: return 0
    
    def just_move(self, xp1):
        try:
            self.ac.move_to_element(self.xpget(xp1)).perform()
            return 1
        except: return 0
        
    def paste_text(self, text):
        try: self.ac.send_keys(text).perform()
        except: pass
        
    def key2elem(self, xpt, key):
        x = self.xplem(xpt)
        if x is not None: 
            ActionChains(self.driver).send_keys_to_element(self.xplem(xpt), getattr(keys, key)).perform()
            return 1
        else:
            print('action not executed, xpath not found')
            return 0
        
    def click_write_enter(self, xpt, text):
        xp = self.xpelem(xpt) if type(xpt) is str else xpt
        if xp is None: return None
        xp.clear()
        if chr(10) not in text:
            self.ac.move_to_element(xp).click().send_keys(text).send_keys(Keys.RETURN).perform()
        else:
            xx = text.split(chr(10))
            for line in xx:
                self.ac.send_keys(line).perform()
                self.ac.key_down(Keys.SHIFT).key_down(Keys.ENTER).key_up(Keys.SHIFT).key_up(Keys.ENTER).perform()
            self.ac.send_keys(Keys.RETURN).perform()
    
    def sidepane_scroll(self, initial = 1, scrolls=35):
        for i in range(1, scrolls):
            self.driver.execute_script("document.getElementById('pane-side').scrollTop={}".format(initial))
            initial = initial + i + 1
        return initial

    def scroll_by_xpath(self, xpt="(//div[@id='main']//div[@class='ItfyB _3nbHh'])[2]"):
        try:
            xp = self.wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
            self.driver.execute_script("arguments[0].scrollIntoView(true);", xp)
            return 1
        except:
            print('WhBase.scroll_by_xpath(xpt) failed to scroll')
            return 0
        
    def msg_sender_dttm(self, n, base="(//div[@id='main']//div[@data-testid='msg-container'])"):
        xpt = base + "[" + str(n) + "]"
        html = self.wait.until(EC.presence_of_element_located((By.XPATH, xpt))).get_attribute('innerHTML')
        sp = BeautifulSoup(html, "lxml")
        x = sp.find_all('div')
        for i in x:
            try:
                st = i['data-pre-plain-text']
                break
            except:pass
        else: return None, None
        try:
            split_point = st.find(']')
            sender = st[split_point+1:].replace(':','').strip(' ')
            dttm = parse(st.upper()[1:split_point])
            return dttm.strftime("%Y-%m-%d %H:%M"), sender
        except: None, None
            
    def xpbulk(self, xpt):
        try:
            if type(xpt) is str: return self.xptext(xpt)
            elif type(xpt) is list: return [self.xptext(n) for n in xpt]
            elif type(xpt) is dict: return {k : [self.xptext(n) for n in v] if type(v) is list else self.xptext(v) for k, v in xpt.items()}
            else: return None
        except:
            return None
    
    def select_chat(self, chat_name):
        if chat_name is not None and chat_name != '':
            x = self.click_write_enter(xpt=self.pane_search, text=chat_name)
            y = self.xptext(self.body_chat_name)
            print('chat body name: ', y)
            if chat_name.lower() in y.lower():
                print('chat selection successful')
                return y
            else: 0
        else: return None
        
    def send_message(self, chat_names, text):
        cn = [chat_names] if type(chat_names) is str else chat_names
        for n in cn:
            x = self.select_chat(n)
            if x is not None or x !=0:
                self.click_write_enter(text)
                print('message send to ', str(n), ' sucessfully')
                tm.sleep(.5)
            else:
                print('message send failed, chat: ', str(n))
    
    def dict_to_df(self, dc, out=None):
        x = dt.datetime.now().strftime('%b%d%Y_%H%M')
        opt = x  + '.csv' if out is None else x + '_' + out + '.csv'
        pd.DataFrame.from_dict(dc).to_csv(opt)
    
    def chat_msg(self, n, base="(//div[@id='main']//div[@data-testid='msg-container'])", xpt_print=False):
        bi = base + '[' + str(n) + ']'
        if self.xpelem(bi) is None: return None
        else:
            self.move_click(bi + "//div[@role='button'][contains(.,'Read more')]")
            dttm, sender = self.msg_sender_dttm(n)
            dc = {'sender' : [sender],
                  'text' : [self.xptext(bi + "//div[@class='_21Ahp']/span[1]/span")],
                  'q_sender' : [self.xptext(bi + "//div[@class='_3pMOs yKTUI']//div[1]/span")],
                  'q_text' : [self.xptext(bi + "//div[@class='_3pMOs yKTUI']//div[2]/span")],
                  'datetime' : [dttm]}
            return dc
        
    def pane_chat(self, n=2, base="(//div[@id='pane-side']//div[@class='_8nE1Y'])"):
        bi = base + '[' + str(n) + ']'
        if self.xpelem(bi) is None: return None
        dc = {'chat_name' : bi + "//div[@class='_21S-L']]",
              'last_text' : bi + "//div[@class='vQ0w7']/span[1]/span[3]",
              'last_sender' : bi + "//div[@class='vQ0w7']/span[1]/span[1]",
              'last_msg_time' : bi + "/div[@class='y_sn4']/div[@class='Dvjym']",
              'new_msg_notif' : bi + "//div[@class='Dvjym']//div[@class='_1pJ9J']/span"}
        for k, v in dc.items():
            v = [v] if type(v) is str else v
            dc[k] = [self.xptext(j) for j in v]
        return dc
    
    def side_pane(self, n=2, base="(//div[@id='pane-side']//div[@class='_8nE1Y'])"):
        bi = base + '[' + str(n) + ']'
        if self.xpelem(bi) is None: return None
        dc = {'chat_name' : bi + "//div[@class='_21S-L']//span[@dir='auto']",
              'last_text' : bi + "//span[@data-testid='last-msg-status']",
              'last_sender' : [bi + "//div[@class='vQ0w7']//span[@dir='auto']", 
                               bi + "//span[@data-testid='status-dblcheck']"],
              'last_msg_time' : bi + "/div[@class='y_sn4']/div[@class='Dvjym']",
              'new_msg_notif' : bi + "//div[@class='Dvjym']//span[@data-testid='icon-unread-count']"}
        for k, v in dc.items():
            v = [v] if type(v) is str else v
            dc[k] = [self.xptext(j) for j in v]
        return dc

In [ ]:
class WATEM(WhBase):
    
    def action_by_duplicate(self, duplicate, base, inx):
        if duplicate == 1: return inx + 1
        elif duplicate == 2:
            self.just_move(base + '[' + str(duplicate-1) + ']')
            tm.sleep(1)
            return 2
        elif duplicate == 3:
            self.scroll_by_xpath(base)
            tm.sleep(1)
            return 3
        else: return 0
        
    def read_ready(self, chatname):
        xptx = self.xptext(self.active_chat_title)
        if xptx is not None and chatname.lower() in xptx.lower():
            print('already selected provided chat (from if): ', chatname)
            self.click("//div[@id='main']//span[@data-testid='down']")
            return 1
        elif xptx is None:
            x = self.select_chat(chatname)
            if chatname.lower() in x.lower():
                print('successfully select chat ', x)
                self.click("//div[@id='main']//span[@data-testid='down']")
                return 1
            else:
                print('chat selection failure, curretly selected: ', x)
                return 0
        else:
            self.driver.refresh
            tm.sleep(20)
            return self.select_chat(chatname)

    def read_action(self, chatname, no_of_sms=20, start_from='last', match_text=None, reply_text=None, forward_to=None, 
                    base="(//div[@id='main']//div[@data-testid='msg-container'])"):
        
        if self.read_ready(chatname) == 0:
            print('read action failed')
            return self
        print('start collecting from older')
        tm.sleep(2)
        i, done, dulicate, nothing, elsecount, x_is_none = 2, 0, 0, 0, 0, 0
        ls, dc = [], defaultdict()
        
        while (no_of_sms>=done and i != 0 and elsecount <= 2):
            x = self.chat_msg(i)
            if x is not None:
                print('x is not none')
                if x['sender'] is None or x['datetime'] is None and x['text'] is None and Nothing<2:
                    Nothing = Nothing + 1
                    i = i + 1
                    print('if')
                elif x['sender'] is not None or x['datetime'] is not None and x['text'] is not None:
                    st = '#'.join(map(str, x['sender'] + x['text'] + x['datetime']))
                    if st in ls:
                        duplicate = duplicate + 1
                        i = self.action_by_duplicate(duplicate, base, i)
                        print('elif if value of duplicate: ', duplicate)
                    else:
                        Nothing, duplicate, done, i = 0, 0, done + 1, i + 1
                        ls.append(st)
                        dc = update_key_value(dc, x)
                        print('elif else value of done: ', done, x)
                else:
                    print('Nothing: ', nothing, ' dup; ', duplicate)
                    print('general scroll to top')
                    i = 2
                    elsecount = elsecount + 1
                    print(x['sender'],x['datetime'],x['text'], 'elsecount: ', elsecount)
            else:
                x_is_none = x_is_none + 1
                if x_is_none > 2: 
                    print('x_is_none: ', x_is_none)
                    break
                self.just_move(base + '[' + str(x_is_none) + ']')
                tm.sleep(1)
                print('x is None , value: ', x_is_none, ' scrolled')
        self.dict_to_df(dc, chatname)
        return dc
    
    def read_from_last(self, chatname, no_of_sms=20, base="(//div[@id='main']//div[@data-testid='msg-container'])"):
        a = self.read_ready(chatname)
        if a == 0 or a is None:
            print('read action failed')
            return self
        print('start collecting from last')
        tm.sleep(2)
        i = ''
        ls, dc = [], defaultdict()
        for n in range(no_of_sms):
            cn = self.xptext(self.active_chat_title)
            if cn is None or chatname.lower() not in cn.lower():
                self.select_chat(chatname)
                tm.sleep(1)
            i = 'last()' if n == 0 else 'last()-' + str(n)
            q = self.xpelem(base + '[' + i + ']')
            try: ActionChains(self.driver).move_to_element(q).perform()
            except:
                print('failed move element')
                self.just_move(base + '[2]')
                tm.sleep(3)
            x = self.chat_msg(i)
            if x is not None:
                if x['sender'] is not None or x['datetime'] is not None and x['text'] is not None:
                    print(n, x)
                    dc = update_key_value(dc, x)
                else:
                    print('all empty else')
            else:
                print('else')
                self.just_move(base + '[3]')
                tm.sleep(3)
                x = self.chat_msg(i)
                if x['sender'] is not None or x['datetime'] is not None and x['text'] is not None:
                    dc = update_key_value(dc, x)
                    yield n, x
                else:
                    print('all emptyy else else')
        self.dict_to_df(dc, chatname + 'read_from_last')
        return dc


In [40]:
def xp(d, xpt, click=False, text=False):
    tm.sleep(.5)
    wait = WebDriverWait(d, 2)
    try: 
        x = wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
        if click == True:
            try: 
                x.click()
                return x, True
            except: 
                print('click failed')
                return x, False
        elif text == True:
            try: 
                print('text: ', x.text)
                return x, x.text
            except: 
                print('no text found')
                return x, None
        else:
            return x
    except: 
        return None

In [ ]:
def xact(driver, xp1, xp2=None, txt=None, click_only=False, paste_only=False):
    wait = WebDriverWait(driver, 3)
    def xpelem(xp):
        try: return wait.until(EC.presence_of_element_located((By.XPATH, xp))) if type(xp) is str else xp
        except: return None
    try: xpelem('/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[1]/span').click()
    except: pass
    y = xpelem(xp1)
    if y is None: 
        print('not found xpath: ', xp1)
        return None
    else:
        ac = ActionChains(driver)
        if xp2 is None and txt is None:
            print("if xp2 is None and txt is None:")
            if click_only==False: ac.move_to_element(y).click().perform()
            else: ac.click(y).perform()
        elif xp2 is not None and txt is None:
            ac.move_to_element(y).click(xpelem(xp2)).perform()
        elif xp2 is not None and txt is not None:
            ac.move_to_element(y).click(xpelem(xp2)).send_keys(txt).perform()
        elif xp2 is None and txt is not None:
            if paste_only==False: ac.move_to_element(y).click().send_keys(txt).perform()
            else: ac.send_keys_to_element(y, txt).perform()
        else: pass
        return 1
    
def msg_action(d, bs="(//div[@id='main']//div[@class='ItfyB _3nbHh'])[last()]",forward=None,reply=None,delete=None):
    
    msgtime = bs + "//div[@class='_2_-To']"
    msgarrow = bs + "//span[@data-testid='down-context']"
    msg_arrow = xact(d, xp1 = msgtime )
    msg_arrow_cl = xact(d, xp1 = msgarrow)
    if delete == True:
        xpt = d, "//div[@class='_2sDI2']//div[contains(text(),'Delete')]"
    elif forward_to is not None:
        select_forward = xact(d, "//div[@class='_2sDI2']//div[contains(text(),'Forward')]")
        forward_button = xact(d, "//div[@class='_2IUvV']//span[@data-testid='forward']")
        search = xact(d, xp1="(//div[contains(@role,'textbox')])[1]", txt= forward_to)
        tm.sleep(.5)
        if xact(d, xp1="(//div[@class='_199zF _3j691 _2N1Gm'])[1]") is None:
            xact(d, xp1='(//*[contains(concat( " ", @class, " " ), concat( " ", "_2N1Gm", " " ))])[1]')
        send_button = xact(d, xp1="//*[@data-testid='send']")
    elif reply_text is not None:
        pass
    else:
        pass

In [41]:
x = WATEM()
d = x.get_driver

WhatsApp


In [44]:
bs = "(//div[@id='main']//div[@class='ItfyB _3nbHh'])[last()]"
forward_to = "halim"
ul = lambda action_text : bs + "//*[@data-testid='" + action_text + "']"

msgtime, msgarrow = str(bs + "//div[@class='_2_-To']"), str(bs + "//span[@data-testid='down-context']")
q = ActionChains(d).move_to_element(xp(d, msgtime)).perform()
tm.sleep(.5)
p = ActionChains(d).click(xp(d, msgarrow)).perform()
ul_item = ul('Forward')
print(ul_item)
elm, click = xp(d, xpt=ul_item, click=True)
if click == True:
    x1 = xp(d, xpt="//div[@class='_2sDI2']//div[contains(text(),'Forward')]", click=True)
    ActionChains(d).send_keys_to_element("(//div[contains(@role,'textbox')])[1]", forward_to).perform()
    x2 = xp(d, xpt="(//div[@class='_199zF _3j691 _2N1Gm'])[1]", click=True)
    x3 = xp(d, xpt="//*[@data-testid='send']", click=True)
    if x3[1] is None: print('forward failed')
    else: print('forward success')

(//div[@id='main']//div[@class='ItfyB _3nbHh'])[last()]//*[@data-testid='Forward']


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
q = xp(d, "(//div[@id='main']//div[@class='ItfyB _3nbHh'])[last()]")
print(q)

In [ ]:
q = x.read_from_last('emergency', 50)
for n in q:
    print(n[0], n[1])

In [46]:
msg_action(d, 'halim')

    


NameError: name 'xact' is not defined

In [ ]:
x.xpds(active_chat_title)

In [ ]:
x.click(active_chat_search)

In [ ]:
def message_read(self, chat_name, no_of_sms=50, base="(//div[@id='main']//div[@data-testid='msg-container'])"):
    Nothing, done, i, duplicate, ls, dc = 0, 2, 2, 0, [], defaultdict()
    c = self.select_chat(chat_name) if chat_name is not None else 1
    if c is None or c == 0: 
        print('could not select chat')
        return None
    self.move_click('/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[1]/span')
    tm.sleep(1)
    while (done < no_of_sms):
        if duplicate>5:
            print('breaking')
            break
        elif duplicate==3:
            self.just_move(base + '[' + str(duplicate-2) + '2]')
            duplicate = duplicate + 1
            i = 2
        else:
            x = self.msgxp(i, base)
            if x is None:
                print('if x is None')
                self.just_move(base + '[2]')
                i = 2
                tm.sleep(1)
            elif x['sender'] is None or x['datetime'] is None and x['text'] is None:
                if Nothing<3:
                    duplicate = duplicate + 1
                else:
                    i = i + 1
                    Nothing = Nothing + 1
            else:
                st = '#'.join(map(str, x['sender'] + x['text'] + x['datetime']))
                if st in ls:
                    duplicate = duplicate + 1
                    i = 2
                else:
                    Nothing, done, i, duplicate = 0, done + 1 , i + 1, 0
                    dc = update_key_value(dc, x)
                    ls.append(st)
    self.dict_to_df(dc, chat_name)
    return dc

In [ ]:
for n in range(0, 11):
    if n == 0: 
        print(x.chat_msg('last()'))
    else:
        print(x.chat_msg('last()-' + str(n)))
        

In [ ]:
def msg_action(d, action_text='forward', forward_to=None, reply_text=None, delete = False, 
               bs = "(//div[@id='main']//div[@class='ItfyB _3nbHh'])[last()]"):
    xact(d, xp1 = str(bs + "//span[@dir='auto']" ),xp2=str(bs + "//span[@data-testid='down-context']"))
    tm.sleep(.5)
    if 'Delete' in action_text:
        rp = xact(d, "//div[@class='_2sDI2']//div[contains(text(),'Delete')]")
    elif 'Reply' in action_text:
        rp = xact(d, "(//div[@class='_2sDI2']//div[contains(text(),'Reply')])[1]")
    elif 'Forward' in action_text:
        select_forward = xact(d, "//div[@class='_2sDI2']//div[contains(text(),'Forward')]")
    else:
        pass

In [ ]:
msg_action(d, "Reply")

In [ ]:
def msg_action(d, action_text='forward', forward_to=None, reply_text=None, delete = False, 
               bs = "(//div[@id='main']//div[@class='ItfyB _3nbHh'])[last()]"):
    msgtime = bs + "//div[@class='_2_-To']"
    msgarrow = bs + "//span[@data-testid='down-context']"
    msg_arrow = xact(d, xp1 = msgtime )
    msg_arrow_cl = xact(d, xp1 = msgarrow)
    if delete == True:
        dele = xact(d,xp1="//*[@data-testid='" + action_text + "']")
    elif forward_to is not None:
        select_forward = xact(d, "//div[@class='_2sDI2']//div[contains(text(),'Forward')]")
        forward_button = xact(d,xp1="//*[@data-testid='" + action_text + "']")
        search = xact(d, xp1="(//div[contains(@role,'textbox')])[1]", txt= forward_to)
        tm.sleep(.5)
        if xact(d, xp1="(//div[@class='_199zF _3j691 _2N1Gm'])[1]") is None:
            xact(d, xp1='(//*[contains(concat( " ", @class, " " ), concat( " ", "_2N1Gm", " " ))])[1]')
        send_button = xact(d, xp1="//*[@data-testid='send']")
    elif reply_text is not None:
        pass
    else:
        pass

In [ ]:
msg_action(d, forward_to='halim')

In [ ]:
delete_for_me = "(//div[contains(@class,'tvf2evcx m0h2a7mj')])[2]"
delete_for_all = "(//div[contains(@class,'tvf2evcx m0h2a7mj')])[3]"

In [ ]:

def xpelm(d, xpt):
    wait = WebDriverWait(d, 3)
    try: return wait.until(EC.presence_of_element_located((By.XPATH, xp))) if type(xp) is str else xp
    except: return None

def click(d, xpt):
    x = xpelm(d, xpt)
    if x is not None:
        try: x.click()
        except:print('click Fail')
    print(xpt, ' - not found')

def move_action(d, xp1, xp2=None, click=False, text=False):
    w = WebDriverWait(d, 2)
    ac = ActionChains(d, 1)
    def xe(xpt):
        tm.sleep(.5)
        try: return w.find_element(By.XPATH, xpt)
        except: return None
    try:
        if xp2 is not None and click==True and text=True:
            ac.move_to_element(xe(xp1)).pause(1).click(xe(xp2)).pause(1).send_keys(text).perform()
        elif xp2 is not None and click==True: 
            ac.move_to_element(xe(xp1)).pause(1).click(xe(xp2)).perform()
        elif text=True and xp2==None and click=False: 
            ac.send_keys_to_element(xe(xp1), text).perform()
        elif xp2 is None and click==True: 
            ac.click(xe(xp1)).perform()
    
    
            
        
chatname = 'emergency'
bs = "(//div[@id='main']//div[@class='ItfyB _3nbHh'])[last()]"
msgtime = xp(bs + "//div[@class='_2_-To']")
msgarrow = bs + "//span[@data-testid='down-context']"
